# How good are LLMs at solving decision theory problems?

In [10]:
import sys
import json
from os import getenv
from pathlib import Path
from datetime import datetime
from openai import OpenAI

# Set the base path
base_path = Path("../")  # One level up from the current working directory

# Add the src/ directory to sys.path using base_path
sys.path.append(str((base_path / "src").resolve()))

from tools.python_math_executor import (
    PYTHON_MATH_EXECUTION_TOOL,
    execute_python
)
from tool_calling import run_tool_call_loop
from yaml_functions import load_yaml

In [11]:
system_prompt = """
You are a decision-analysis expert with deep knowledge of decision theory, expected utility, and probabilistic reasoning. 
Your goal is to evaluate decision problems objectively and recommend the strategy that maximizes expected utility, based on the information provided.
"""

short_objective = """
## Objective

Given the problem description above, your task is to identify the rational decision(s) at each point in the process. Use the information provided to evaluate the options and explain your reasoning clearly. Return your final recommendation based on sound analysis.
"""

long_objective = """
## Objective

Given the problem above, your task is to determine the optimal decision strategy using rational, quantitative reasoning.

Your response should include:

- **Modeling the Problem**  
  Identify the decision points, uncertainties (chance events), possible outcomes, and the utilities or payoffs associated with each outcome. Include any costs that affect final payoffs.

- **Computing Expected Utilities**  
  For each possible decision or strategy, calculate the expected utility by combining the relevant probabilities and payoffs. Clearly show how the values are derived.

- **Comparing Strategies**  
  Evaluate and compare the expected utilities of all possible strategies or policies. Determine which strategy yields the highest expected utility.

- **Recommending the Optimal Policy**  
  Based on your analysis, state the strategy that maximizes expected utility. If the problem involves observations or tests, describe what to do for each possible result.

- **Documenting the Reasoning**  
  Show all relevant calculations and briefly explain each step in your reasoning. Make the rationale behind your recommendation transparent and verifiable.
"""

In [12]:
problem = load_yaml(base_path / "decision_problems/oil_field_purchase/problem.yaml")["long_version"]

objective = short_objective

prompt_str = problem + "\n" + objective

In [13]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=getenv("OPENROUTER_API_KEY")
)

# Messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt_str}
]


MODEL = "gpt-4o-mini"

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # To allow for multiple runs

In [14]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=getenv("OPENROUTER_API_KEY")  # Make sure this env variable is set
)

response = client.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

To address the Oil Field Investment Decision Problem, we will analyze whether to perform the Porosity Test and subsequently analyze the decision of whether to buy the oil field based on the potential outcomes.

### Step 1: Evaluating the Test Option

#### Expected Utility with Test

To calculate the expected utility when performing the test, we will consider the two results of the test: **Pass** and **Fail**, and use the law of total probability to find the expected utilities for each.

#### Calculating P(R) 

Using the prior probabilities and the test reliability:
- **P(Pass)**: 

\[
P(Pass) = P(Pass|High) \cdot P(High) + P(Pass|Medium) \cdot P(Medium) + P(Pass|Low) \cdot P(Low) 
\]
\[
= (0.90 \cdot 0.30) + (0.65 \cdot 0.40) + (0.15 \cdot 0.30) 
\]
\[
= 0.27 + 0.26 + 0.045 = 0.575
\]

- **P(Fail)**: 

\[
P(Fail) = P(Fail|High) \cdot P(High) + P(Fail|Medium) \cdot P(Medium) + P(Fail|Low) \cdot P(Low) 
\]
\[
= (0.10 \cdot 0.30) + (0.35 \cdot 0.40) + (0.85 \cdot 0.30) 
\]
\[
= 0.03 + 0.1